In [31]:
%reload_ext autoreload
%autoreload 2

import os
import sys
while os.path.split(os.getcwd())[1] != 'RecSysChallenge2023-Team':
    os.chdir('..')
sys.path.insert(1, os.getcwd())

import numpy as np

In [32]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_user_wise
from Evaluation.Evaluator import EvaluatorHoldout

import Daniele.Utils.MyDataManager as dm
import Daniele.Utils.MatrixManipulation as mm

URMv = dm.getURMviews()
URMo = dm.getURMopen()
#URM_all = mm.defaultExplicitURM(urmv=URMv, urmo=URMo, normalize=False, add_aug=False)
URM_all = URMo + URMv
URM_all.data = np.ones(len(URM_all.data)) #Binaria

ICMt=dm.getICMt()
ICMl=dm.getICMl()

URM_train_validation, URM_test = split_train_in_two_percentage_user_wise(URM_all, train_percentage = 0.85)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)


In [33]:
from lightfm import LightFM

model = LightFM(
    loss='warp',
    #item_alpha=0.1,
    #user_alpha=0.1,
)
%time
model.fit(URM_train_validation, epochs=1, verbose=True)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


Epoch: 100%|██████████| 1/1 [00:01<00:00,  1.88s/it]


In [34]:
from lightfm.evaluation import precision_at_k

print("Train precision: %.2f" % precision_at_k(model, URM_train_validation, k=10).mean())
print("Test precision: %.2f" % precision_at_k(model, URM_test, k=10).mean())

Train precision: 0.11
Test precision: 0.02


In [8]:
evaluator_test = EvaluatorHoldout(URM_test, [10])
results, result_string = evaluator_test.evaluateRecommender(model)

EvaluatorHoldout: Ignoring 318 ( 0.8%) Users that have less than 1 test interactions


AttributeError: 'LightFM' object has no attribute 'get_URM_train'

In [ ]:
results